# Data preprocessing for Fusion and Model

Importing Libraries

In [1]:
# Essentials
import pandas as pd
import numpy as np

# Processing
from sklearn.preprocessing import StandardScaler

# Model

# Visualization

# Warnings
import warnings
warnings.filterwarnings('ignore')

Loading Datasets

In [ ]:
# Clinical Datasets
African = pd.read_csv(r"C:\Users\dahab\OneDrive\Desktop\T2D-Prediction-System--Data-Fusion-for-Enhanced-Decision-Making\processed_datasets\clinical\African_pro.csv")
Bangladesh = pd.read_csv(r"C:\Users\dahab\OneDrive\Desktop\T2D-Prediction-System--Data-Fusion-for-Enhanced-Decision-Making\processed_datasets\clinical\Bangladesh_pro.csv")
Iraq = pd.read_csv(r"C:\Users\dahab\OneDrive\Desktop\T2D-Prediction-System--Data-Fusion-for-Enhanced-Decision-Making\processed_datasets\clinical\Iraq_pro.csv")

# Genetic Datasets
inter_genetic = pd.read_csv(r"C:\Users\dahab\OneDrive\Desktop\T2D-Prediction-System--Data-Fusion-for-Enhanced-Decision-Making\processed_datasets\genetic\inter_genetic_dataset.csv")
normal_genetic = pd.read_csv(r"C:\Users\dahab\OneDrive\Desktop\T2D-Prediction-System--Data-Fusion-for-Enhanced-Decision-Making\processed_datasets\genetic\normal_genetic_dataset.csv")

In [4]:
inter_genetic.head()

,STUDY,DISEASE_DESCRIPTION,REGION,CHR_ID,CHR_POS,MAPPED_GENE,UPSTREAM_GENE_ID,DOWNSTREAM_GENE_ID,UPSTREAM_GENE_DISTANCE,DOWNSTREAM_GENE_DISTANCE,...,CASE_PERCENTAGE,CI_LOWER_BOUND,CI_UPPER_BOUND,CI_RANGE,SNPS_PASSING_QC,PLATFORM_AFFYMETRIX,PLATFORM_AFFYMETRIX_ILLUMINA,PLATFORM_ILLUMINA,IMPUTED_ENCODED,RISK_ALLELE_ENCODED
0,[-1.47616668e-02 5.09463213e-02 2.69961264e-...,[ 1.58013031e-01 9.36319865e-03 -6.81496318e-...,[ 3.05748135e-01 -2.47293934e-01 -1.75472543e-...,8,21853920,[ 1.15677923e-01 4.50637192e-02 -5.24731204e-...,168546,147443,41563,54953,...,18.674502,0.46,0.56,0.09,6494962,0,1,0,1,0
1,[-1.47616668e-02 5.09463213e-02 2.69961264e-...,[ 1.58013031e-01 9.36319865e-03 -6.81496318e-...,[ 3.67390394e-01 -3.92685801e-01 1.06864296e-...,12,5284163,[ 2.10065857e-01 -2.48143420e-01 -4.31279140e-...,256115,185652,39964,147945,...,18.674502,0.46,0.56,0.09,6494962,0,1,0,1,0
2,[ 1.53467417e-01 -1.21334471e-01 7.48024210e-...,[ 1.58013031e-01 9.36319865e-03 -6.81496318e-...,[ 2.82850683e-01 -1.11950912e-01 2.99117025e-...,8,107059816,[ 3.28507662e-01 -2.92603046e-01 -2.57462531e-...,174429,254146,289572,113384,...,22.767117,1.34,2.08,0.74,6906962,0,1,0,1,1
3,[ 4.83314842e-02 1.37150949e-02 -2.07627401e-...,[-1.53973475e-01 1.47206217e-01 -1.56758264e-...,[ 2.93079883e-01 -1.66859761e-01 -1.55704990e-...,7,15013235,[ 3.02324921e-01 -1.51797831e-01 -1.33085012e-...,225816,187546,27161,187082,...,17.630698,1.04,1.09,0.05,2500000,0,1,0,1,2
4,[ 4.83314842e-02 1.37150949e-02 -2.07627401e-...,[-1.53973475e-01 1.47206217e-01 -1.56758264e-...,[ 2.77950853e-01 -2.00821161e-01 -7.99589884e-...,9,22134095,[ 4.64857012e-01 -1.08726546e-01 -2.27923006e-...,240498,176399,5992,312729,...,17.630698,1.15,1.22,0.07,2500000,0,1,0,1,0


Checking Target Columns

In [64]:
print(African.columns, "\n")
print(Bangladesh.columns, "\n")
print(Iraq.columns, "\n")
print(inter_genetic.columns, "\n")
print(normal_genetic.columns)

Index(['Patient number', 'Cholesterol', 'Glucose', 'HDL Chol',
       'Chol/HDL ratio', 'Age', 'Gender', 'Height', 'Weight', 'BMI',
       'Systolic BP', 'Diastolic BP', 'waist', 'hip', 'Waist/hip ratio',
       'Diabetes', 'BMI Category'],
      dtype='object') 

Index(['age', 'pulse_rate', 'systolic_bp', 'diastolic_bp', 'glucose', 'height',
       'weight', 'bmi', 'family_diabetes', 'hypertensive',
       'family_hypertension', 'cardiovascular_disease', 'stroke',
       'gender_Encoded', 'diabetic_Encoded'],
      dtype='object') 

Index(['Age', 'Urea', 'Cr', 'HbA1c', 'Chol', 'TG', 'HDL', 'LDL', 'VLDL', 'BMI',
       'Gender_Encoded', 'Class_D', 'Class_N', 'Class_P'],
      dtype='object') 

Index(['STUDY', 'DISEASE_DESCRIPTION', 'REGION', 'CHR_ID', 'CHR_POS',
       'MAPPED_GENE', 'UPSTREAM_GENE_ID', 'DOWNSTREAM_GENE_ID',
       'UPSTREAM_GENE_DISTANCE', 'DOWNSTREAM_GENE_DISTANCE', 'SNPS', 'MERGED',
       'GENOMIC_CONTEXT', 'INTERGENIC', 'RISK_ALLELE_FREQUENCY', 'PVALUE',
       'P

Uniting Target Columns' name

In [ ]:
African.rename(columns={
    'Diabetes': 'T2D',
}, inplace=True)

Bangladesh.rename(columns={
    'diabetic_Encoded': 'T2D',
}, inplace=True)

Iraq.rename(columns={
    'Class_Encoded': 'T2D',
}, inplace=True)

# Add a new column 'T2D' with all values set to 1
inter_genetic['T2D'] = 1

# Add a new column 'T2D' with all values set to 1
normal_genetic['T2D'] = 1

Checking Data Types

In [67]:
print(African.dtypes, '\n')
print(Bangladesh.dtypes, '\n')
print(Iraq.dtypes, '\n')
print(inter_genetic.dtypes, '\n')
print(normal_genetic.dtypes, '\n')

Patient number       int64
Cholesterol          int64
Glucose              int64
HDL Chol             int64
Chol/HDL ratio     float64
Age                  int64
Gender               int64
Height               int64
Weight               int64
BMI                float64
Systolic BP          int64
Diastolic BP         int64
waist                int64
hip                  int64
Waist/hip ratio    float64
T2D                  int64
BMI Category         int64
dtype: object 

age                         int64
pulse_rate                  int64
systolic_bp                 int64
diastolic_bp                int64
glucose                   float64
height                    float64
weight                    float64
bmi                       float64
family_diabetes             int64
hypertensive                int64
family_hypertension         int64
cardiovascular_disease      int64
stroke                      int64
gender_Encoded              int64
T2D                         int64
dtype: object 


Feature Scalling

In [ ]:
# Drop target labels and scale features
scaler = StandardScaler()

X_african = scaler.fit_transform(African.drop(columns=['T2D']))
X_bangladesh = scaler.fit_transform(Bangladesh.drop(columns=['T2D']))
X_iraq = scaler.fit_transform(Iraq.drop(columns=['T2D_D', 'T2D_N', 'T2D_P']))
X_inter_genetic = scaler.fit_transform(inter_genetic.drop(columns=['T2D']))
X_normal_genetic = scaler.fit_transform(normal_genetic.drop(columns=['T2D']))

Splitting Data

Sub-Networks for Each Dataset

Fusion

Output Layer

Compiling Model

Training Model

Model Evaluation